# Notebook 06v2 - Fully Built Macro Research Pipeline
Everything prebuilt, looped, and standardized so you don’t have to handcraft charts per metric.

Goal of 06v2:
- Define a METRICS registry
- Generate the same set of charts for every metric
- Produce a clean current snapshot export
- Keep it readable enough for a blog/video walkthrough

In [1]:
# imports

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

from macro_utils.transforms import zscore
from macro_utils.utils import (
    build_transformed_dataset,
    prepare_plot_df,
)
from macro_utils.regimes import build_macro_regimes
from macro_utils.events import event_window

DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_OUTPUTS = PROJECT_ROOT / "data" / "outputs"
DATA_OUTPUTS.mkdir(parents=True, exist_ok=True)

In [2]:
# load data

monthly = pd.read_csv(
    DATA_RAW / "fred_monthly.csv",
    index_col=0,
    parse_dates=True
)

In [3]:
# canonical transformed dataset

dash = build_transformed_dataset(monthly)
dash.tail()

c:\Users\JR\OneDrive\Mini PC\FRED\macro_utils\transforms.py:5: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return series.pct_change(periods) * 100
c:\Users\JR\OneDrive\Mini PC\FRED\macro_utils\transforms.py:8: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return series.pct_change(1) * 100


,GDP_YoY,CPI_YoY,CPI_MoM,UNRATE,FEDFUNDS
DATE,,,,,
2025-07-31,5.365421,2.731801,0.196579,4.3,4.33
2025-08-31,5.365421,2.939220,0.382452,4.3,4.33
2025-09-30,5.365421,3.022700,0.310486,4.4,4.22
2025-11-30,4.257835,2.711969,0.204397,4.5,3.88
2025-12-31,4.257835,2.653312,0.307355,4.4,3.72


In [4]:
# macro regimes

dash = build_macro_regimes(dash)
dash["Macro_Regime"].value_counts()

Macro_Regime
Expansion / Inflationary / Easing           112
Expansion / Inflationary / Tightening       108
Expansion / Disinflationary / Tightening    103
Expansion / Disinflationary / Easing         78
Contraction / Disinflationary / Easing       17
Contraction / Inflationary / Easing           1
Name: count, dtype: int64

In [5]:
# metric registry

METRICS = {
    "Inflation (CPI YoY)": {"column": "CPI_YoY", "label": "YoY %"},
    "Inflation (CPI MoM)": {"column": "CPI_MoM", "label": "MoM %"},
    "Growth (GDP YoY)": {"column": "GDP_YoY", "label": "YoY %"},
    "Unemployment Rate": {"column": "UNRATE", "label": "%"},
    "Policy Rate (Fed Funds)": {"column": "FEDFUNDS", "label": "%"},
}

In [6]:
# time series chart for every metric

plot_df = prepare_plot_df(dash)

for name, meta in METRICS.items():
    col = meta["column"]

    fig = px.line(
        plot_df,
        x="DATE",
        y=col,
        title=f"{name} — Time Series",
    )
    fig.update_layout(height=400)
    fig.show()

In [7]:
# Distribution Charts (by macro regime)

for name, meta in METRICS.items():
    col = meta["column"]

    tmp = dash[[col, "Macro_Regime"]].dropna().copy()
    tmp[col] = pd.to_numeric(tmp[col], errors="coerce")
    tmp = tmp.dropna()

    fig = px.box(
        tmp,
        x="Macro_Regime",
        y=col,
        points="outliers",
        title=f"{name} — Distribution by Macro Regime",
    )
    fig.update_layout(height=450)
    fig.show()

In [8]:
# Regime Conditional Summary

num_cols = [m["column"] for m in METRICS.values()]

summary = (
    dash
    .groupby("Macro_Regime")[num_cols]
    .agg(["mean", "std"])
)

summary

CPI_YoY             CPI_MoM  \
                                              mean       std      mean   
Macro_Regime                                                             
Contraction / Disinflationary / Easing   -0.130521  1.035789  0.014645   
Contraction / Inflationary / Easing       3.731058       NaN -0.859844   
Expansion / Disinflationary / Easing      1.749719  0.459090  0.151636   
Expansion / Disinflationary / Tightening  1.602413  0.688677  0.118909   
Expansion / Inflationary / Easing         3.524492  1.110600  0.297863   
Expansion / Inflationary / Tightening     3.717800  1.573232  0.297998   

                                                     GDP_YoY            \
                                               std      mean       std   
Macro_Regime                                                             
Contraction / Disinflationary / Easing    0.623738 -2.722058  2.245557   
Contraction / Inflationary / Easing            NaN -0.726120       NaN   
Expansion / Disinflationary / Easing      0.179238  4.281563  1.625839   
Expansion / Disinflationary / Tightening  0.211631  4.458728  1.084269   
Expansion / Inflationary / Easing         0.203355  5.042124  2.433090   
Expansion / Inflationary / Tightening     0.261635  6.582467  2.066896   

                                            UNRATE            FEDFUNDS  \
                                              mean       std      mean   
Macro_Regime                                                             
Contraction / Disinflationary / Easing    9.476471  2.032095  0.151176   
Contraction / Inflationary / Easing       6.500000       NaN  0.970000   
Expansion / Disinflationary / Easing      5.796154  1.480299  1.897692   
Expansion / Disinflationary / Tightening  5.584466  1.772765  1.754078   
Expansion / Inflationary / Easing         6.123214  1.580739  3.161071   
Expansion / Inflationary / Tightening     4.618519  0.857746  4.224907   

                                                    
                                               std  
Macro_Regime                                        
Contraction / Disinflationary / Easing    0.080147  
Contraction / Inflationary / Easing            NaN  
Expansion / Disinflationary / Easing      1.900108  
Expansion / Disinflationary / Tightening  2.037167  
Expansion / Inflationary / Easing         2.011209  
Expansion / Inflationary / Tightening     1.799680

In [9]:
# adding counts

counts = dash.groupby("Macro_Regime").size().rename("n_obs")

summary_with_counts = summary.copy()
summary_with_counts[("meta", "n_obs")] = counts

summary_with_counts

CPI_YoY             CPI_MoM  \
                                              mean       std      mean   
Macro_Regime                                                             
Contraction / Disinflationary / Easing   -0.130521  1.035789  0.014645   
Contraction / Inflationary / Easing       3.731058       NaN -0.859844   
Expansion / Disinflationary / Easing      1.749719  0.459090  0.151636   
Expansion / Disinflationary / Tightening  1.602413  0.688677  0.118909   
Expansion / Inflationary / Easing         3.524492  1.110600  0.297863   
Expansion / Inflationary / Tightening     3.717800  1.573232  0.297998   

                                                     GDP_YoY            \
                                               std      mean       std   
Macro_Regime                                                             
Contraction / Disinflationary / Easing    0.623738 -2.722058  2.245557   
Contraction / Inflationary / Easing            NaN -0.726120       NaN   
Expansion / Disinflationary / Easing      0.179238  4.281563  1.625839   
Expansion / Disinflationary / Tightening  0.211631  4.458728  1.084269   
Expansion / Inflationary / Easing         0.203355  5.042124  2.433090   
Expansion / Inflationary / Tightening     0.261635  6.582467  2.066896   

                                            UNRATE            FEDFUNDS  \
                                              mean       std      mean   
Macro_Regime                                                             
Contraction / Disinflationary / Easing    9.476471  2.032095  0.151176   
Contraction / Inflationary / Easing       6.500000       NaN  0.970000   
Expansion / Disinflationary / Easing      5.796154  1.480299  1.897692   
Expansion / Disinflationary / Tightening  5.584466  1.772765  1.754078   
Expansion / Inflationary / Easing         6.123214  1.580739  3.161071   
Expansion / Inflationary / Tightening     4.618519  0.857746  4.224907   

                                                    meta  
                                               std n_obs  
Macro_Regime                                              
Contraction / Disinflationary / Easing    0.080147    17  
Contraction / Inflationary / Easing            NaN     1  
Expansion / Disinflationary / Easing      1.900108    78  
Expansion / Disinflationary / Tightening  2.037167   103  
Expansion / Inflationary / Easing         2.011209   112  
Expansion / Inflationary / Tightening     1.799680   108

In [10]:
# event study setup (contractions)

event_dates = dash.index[
    dash["Macro_Regime"].shift(1).str.contains("Expansion", na=False) &
    dash["Macro_Regime"].str.contains("Contraction", na=False)
]

event_dates[:10]

DatetimeIndex(['2008-10-31', '2020-04-30'], dtype='datetime64[ns]', name='DATE', freq=None)

In [11]:
# event studies for all metrics

WINDOW = 12

for name, meta in METRICS.items():
    col = meta["column"]
    series = dash[col].dropna()

    windows = event_window(series, event_dates, window=WINDOW)

    print(f"{name}: {len(windows)} event windows")

    if windows.empty or len(windows) < 3:
        print(f"Skipping {name} (insufficient event windows)")
        continue

    mean_path = windows.mean()
    p10 = windows.quantile(0.10)
    p90 = windows.quantile(0.90)

    fig = go.Figure()

    # individual paths
    for i in range(len(windows)):
        fig.add_trace(go.Scatter(
            x=windows.columns,
            y=windows.iloc[i].values,
            mode="lines",
            opacity=0.15,
            line=dict(width=1),
            showlegend=False
        ))

    # 10–90% band
    fig.add_trace(go.Scatter(
        x=mean_path.index, y=p90.values,
        mode="lines", line=dict(width=0),
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=mean_path.index, y=p10.values,
        fill="tonexty",
        mode="lines",
        line=dict(width=0),
        opacity=0.25,
        name="10–90% band"
    ))

    # mean
    fig.add_trace(go.Scatter(
        x=mean_path.index,
        y=mean_path.values,
        mode="lines",
        line=dict(width=3),
        name="Mean"
    ))

    fig.add_vline(x=0, line_width=2, line_dash="dash")

    fig.update_layout(
        title=f"{name} — Event Study Around Contractions",
        xaxis_title="Months Relative to Event",
        yaxis_title=meta["label"],
        height=500
    )

    fig.show()


Inflation (CPI YoY): 2 event windows
Skipping Inflation (CPI YoY) (insufficient event windows)
Inflation (CPI MoM): 2 event windows
Skipping Inflation (CPI MoM) (insufficient event windows)
Growth (GDP YoY): 2 event windows
Skipping Growth (GDP YoY) (insufficient event windows)
Unemployment Rate: 2 event windows
Skipping Unemployment Rate (insufficient event windows)
Policy Rate (Fed Funds): 2 event windows
Skipping Policy Rate (Fed Funds) (insufficient event windows)


- Event studies around contractions are sample-limited.
- Under strict regime definitions, only a small number of clean events exist.

In [12]:
# current regime snapshot

latest_date = dash.index.max()
current_regime = dash.loc[latest_date, "Macro_Regime"]

latest_date, current_regime

(Timestamp('2025-12-31 00:00:00'), 'Expansion / Inflationary / Easing')

In [13]:
# export current snapshot

def _scalar(x):
    if hasattr(x, "iloc"):
        return x.iloc[0]
    return x

snapshot = pd.DataFrame([{
    "date": latest_date.date().isoformat(),
    "macro_regime": current_regime,
    "cpi_yoy": float(_scalar(dash.loc[latest_date, "CPI_YoY"])),
    "cpi_mom": float(_scalar(dash.loc[latest_date, "CPI_MoM"])),
    "gdp_yoy": float(_scalar(dash.loc[latest_date, "GDP_YoY"])),
    "unrate": float(_scalar(dash.loc[latest_date, "UNRATE"])),
    "fedfunds": float(_scalar(dash.loc[latest_date, "FEDFUNDS"])),
}])

snapshot

,date,macro_regime,cpi_yoy,cpi_mom,gdp_yoy,unrate,fedfunds
0,2025-12-31,Expansion / Inflationary / Easing,2.653312,0.307355,4.257835,4.4,3.72


In [14]:
# save

snap_path = DATA_OUTPUTS / "current_macro_snapshot.csv"
snapshot.to_csv(snap_path, index=False)

snap_path

WindowsPath('c:/Users/JR/OneDrive/Mini PC/FRED/data/outputs/current_macro_snapshot.csv')

In [15]:
# save full dashboard data

dash_path = DATA_OUTPUTS / "macro_dashboard_table.csv"
dash.to_csv(dash_path)

dash_path

WindowsPath('c:/Users/JR/OneDrive/Mini PC/FRED/data/outputs/macro_dashboard_table.csv')